In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import math
import joblib

In [3]:
df = pd.read_csv("classifier_dataset.csv")

In [4]:
df[f"hr-predict-Temp"] = df["Temperature"].shift(-26, axis=0)
df[f"hr-predict-Humidity"] = df["Humidity"].shift(-26, axis=0)
df[f"hr-predict-WNDSPD"] = df["Wind_Speed"].shift(-26, axis=0)
df[f"hr-predict-WNDDIR"] = df["Wind_Direction"].shift(-26, axis=0)
df[f"hr-predict-Pressure"] = df["Pressure"].shift(-26, axis=0)


In [5]:
for x in np.arange(1, 109):
    shift_distance = x*26
    df[f"{x}-hrs-ago-Temp"] = df["Temperature"].shift(shift_distance, axis=0)
    df[f"{x}-hrs-ago-Humidity"] = df["Humidity"].shift(shift_distance, axis=0)
    df[f"{x}-hrs-ago-WNDSPD"] = df["Wind_Speed"].shift(shift_distance, axis=0)
    df[f"{x}-hrs-ago-WNDDIR"] = df["Wind_Direction"].shift(shift_distance, axis=0)
    df[f"{x}-hrs-ago-Pressure"] = df["Pressure"].shift(shift_distance, axis=0)

In [6]:
without_previous_data = 108 * 26
without_future_data = -1 * 26

df = df.iloc[without_previous_data:without_future_data,:]

In [7]:
features = ['Timestamp', 'Temperature', 'Humidity', 'Wind_Speed',
    'Wind_Direction', 'Pressure', 'Year', 'Day sin', 'Day cos', 'Year sin', 'Year cos',
    'Latitude', 'Longitude']

for x in np.arange(1, 109):
    features.append(f"{x}-hrs-ago-Temp")
    features.append(f"{x}-hrs-ago-Humidity")
    features.append(f"{x}-hrs-ago-WNDSPD")
    features.append(f"{x}-hrs-ago-WNDDIR")
    features.append(f"{x}-hrs-ago-Pressure")

In [ ]:
selected_features = df[features]

y = df[['hr-predict-Temp']]
# y = df[['hr-predict-Humidity']]
# y = df[['hr-predict-WNDSPD']]
# y = df[['hr-predict-WNDDIR']]
# y = df[['hr-predict-Pressure']] 

In [8]:
y = y.values.reshape(-1,1)
y.shape

(1154333, 1)

In [9]:
length = math.floor(len(df)*0.8)
X_train = selected_features[:length]
y_train = y[:length]
X_test = selected_features[length:]
y_test = y[length:]

In [10]:
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [11]:
model = joblib.load("time_cycles/hr_temp_predict_model.h5")
# model = joblib.load("time_cycles/hr_humidity_predict_model.h5")
# model = joblib.load("time_cycles/hr_wind_sp_predict_model.h5")
# model = joblib.load("time_cycles/hr_wind_dr_predict_model.h5")
# model = joblib.load("time_cycles/hr_pressure_predict_model.h5")

In [15]:
predictions = model.predict(X_test_scaled[0:10])

In [21]:
print(y_scaler.inverse_transform(predictions)[0])
print(y_scaler.inverse_transform(y_test_scaled[0:10])[0])

[288.88744887]
[292.72]
